In [1]:
import syft as sy
from syft.frameworks.torch.tensors.interpreters import PointerTensor
from syft.frameworks.torch.tensors.decorators import LoggingTensor
import sys
import torch
hook = sy.TorchHook(torch)
from torch.nn import Parameter
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Tensors

In [2]:
bob = sy.VirtualWorker(hook, id="bob")
bob

<VirtualWorker id:bob #tensors:0>

In [3]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

In [4]:
xp = x.send(bob)
yp = y.send(bob)
xp

(Wrapper)>[PointerTensor | me:71245146895 -> bob:8287481223]

In [5]:
bob._objects

{8287481223: tensor([1, 2, 3, 4, 5]), 7217679164: tensor([1, 1, 1, 1, 1])}

In [6]:
z = xp + yp
bob._objects

{8287481223: tensor([1, 2, 3, 4, 5]),
 7217679164: tensor([1, 1, 1, 1, 1]),
 16450610551: tensor([2, 3, 4, 5, 6])}

In [7]:
bob == z.location, z.location

(True, <VirtualWorker id:bob #tensors:3>)

In [8]:
z.owner # we are VirtualWorker too

<VirtualWorker id:me #tensors:0>

In [9]:
z.get()

tensor([2, 3, 4, 5, 6])

# Federated Learning

In [10]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [11]:
x = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
y = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [12]:
x_bob, x_alice = x[:2], x[2:]
y_bob, y_alice = y[:2], y[2:]

x_bob, x_alice = x_bob.send(bob), x_alice.send(alice)
y_bob, y_alice = y_bob.send(bob), y_alice.send(alice)

datasets = [(x_bob,y_bob),(x_alice,y_alice)]

In [13]:
model = nn.Linear(2,1)
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [14]:
for i in range(10):
    for x, y_hat in datasets:
        model.send(x.location) # NEW
        opt.zero_grad() # erase previous gradients (if they exist)
        y = model(x)
        loss = ((y - y_hat)**2).sum()
        loss.backward()
        opt.step()
        model.get() # NEW
        print('{:.4}'.format(loss.get().data.numpy()))

0.1653
3.434
1.146
0.6314
0.7376
0.3287
0.4318
0.1886
0.2514
0.109
0.1463
0.063
0.08528
0.03641
0.04975
0.02105
0.02906
0.01217
0.01699
0.007038


# Encrypted Calculations

In [15]:
bill = sy.VirtualWorker(hook, id="bill")

In [17]:
x = torch.tensor([25])
enc_x = x.share(bob, alice, bill)
dec_x = enc_x.get()
dec_x

tensor([25])

In [18]:
trusted_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [20]:
x = torch.tensor([25]).share(bob, alice, crypto_provider=trusted_provider)
y = torch.tensor([5]).share(bob, alice, crypto_provider=trusted_provider)

In [21]:
z = x * y
z.get()

tensor([125])